# Bioinformatics - Programming Assignment – 3
## Debayan Saha - 2019113002

In [2]:
import numpy as np

In [3]:
def populate_matrix(File):
    matrixFile = open(File, "r")
    lines = matrixFile.readlines()
    matrixFile.close()
    dictaa = {}
    aminoacidstring = lines[0]
    aminoacidstring = aminoacidstring.split()
    
    i = 1
    while i <= (len(lines)-1):
        row = lines[i]
        row = row.split()

        j = 1
        for character in row[1:len(lines)]:
            dictaa[aminoacidstring[i-1],aminoacidstring[j-1]] = character # i,j changes (row, column) for keys
            j+=1
        i+=1

    return(dictaa) 

In [4]:
def seq_inp(path):
    input_f = open(path, "r")
    input_f.readline()

    seq=""

    for line in input_f:
        seq += line.replace("\n", "")
    return seq

In [5]:
dna_seq1 = seq_inp("./seq1.fasta")
dna_seq2 = seq_inp("./seq2.fasta")
prot_seq1 = seq_inp("./protein1.fasta")
prot_seq2 = seq_inp("./protein2.fasta")

In [6]:
blo_dict = populate_matrix("./BLOSUM62.txt")
dna_dict = populate_matrix("./DNAfull.txt")

### Question 1 - Global Alignment

* Default gap penalty = 10
* Score Matirx used = BLOSUM62 - protein / DNAfull - DNA

In [15]:
def global_align(seq1, seq2, gap_penalty, blosum_dict):

    smatrixToDict = {}
    direction = 0 
    maxLocation = 0,0
    
    smatrix = np.zeros((len(seq1)+2, len(seq2)+2), dtype = np.object)
    
    toprow = smatrix[0]
    seq2 = " "+ " " + seq2
    seq1 = " "+ seq1
    for i in range(len(seq2)):
        toprow[i] = seq2[i] 

    for k in range(len(seq1)): 
        smatrix[k+1][0] = seq1[k]

    for rowi in range(1, len(seq1)+1): 
        
        for colj in range(1, len(seq2)):  

            blosumScoreKey = (smatrix[rowi][0],smatrix[0][colj])
            if(rowi == 1 and colj ==1):
                continue
        
#             print(smatrix)
            
            if(rowi == 1):
                scorefromleft = int(smatrix[1][colj-1]) -(gap_penalty)
                maxof4scores = scorefromleft
                direction = 3
            
            elif(colj == 1):
                scorefromabove = int(smatrix[rowi-1][1]) -(gap_penalty)
                maxof4scores = scorefromabove
                direction = 1
                
            else:
                scorefromabove = int(smatrix[rowi-1][colj]) -(gap_penalty)
                scorefromleft = int(smatrix[rowi][colj-1]) -(gap_penalty)
                scorefromdiag= (smatrix[rowi-1][colj-1]) + int(blosum_dict.get(blosumScoreKey))

                maxof4scores = max(scorefromabove, scorefromleft, scorefromdiag)
                
                if(maxof4scores == scorefromabove): # gap goes in seq1 horiz aa
                    direction = 1
                if(maxof4scores == scorefromleft): # gap goes in seq2 vert aa 
                    direction = 3
                if(maxof4scores == scorefromdiag):
                    direction = 2

            smatrix[rowi][colj] = maxof4scores
                
            smatrixToDict[rowi, colj] = direction, maxof4scores 
                       
            maxLocation = rowi, colj    
 
    maxScore = smatrix[maxLocation]           
    location = maxLocation       
    
#     print(smatrix)
    directionANDscore = smatrixToDict[location]
    direction, score = (directionANDscore[0], directionANDscore[1])
    newseq1 = []
    newseq2 = []
     
     
    while 1:      
        if(direction == 1):  #score from above
            newlocation = ((location[0])-1, location[1])  #location = [row, column] of score
            newseq2.append("_")
            newseq1.append(smatrix[location[0]][0])  #smatrix[locationtuple(row, col)][0]
                                                         
        if(direction == 3):    #score from left
            newseq1.append("_")   
            newlocation = (location[0], (location[1])-1)
            newseq2.append(smatrix[0][location[1]])             

        if(direction == 2): #score from diagonal                      
            newlocation = ((location[0])-1, (location[1])-1)
            newseq1.append(smatrix[location[0]][0])                    
            newseq2.append(smatrix[0][location[1]])  

#         print(score)
#         print(newseq2)
        location = newlocation
        if(location[0] == 1 and location[1] == 1):
            break
        directionANDscore = smatrixToDict[location]
        score = (directionANDscore[1]) 
        direction = directionANDscore[0]
    
    newseq1 = "".join(newseq1[::-1])
    newseq2 = "".join(newseq2[::-1])
    print("Max Score: ", maxScore)
#     print("Location: ", maxLocation)
    
    print("\nGlobal Alignment: ")
    print(newseq1)
    print(newseq2)


In [16]:
global_align(dna_seq1, dna_seq2, 10, dna_dict)

Max Score:  -435

Global Alignment: 
_TTCAG_TAAACATTAC_GA_TTGTGCA_AAAGTAAAGAGTGTGTTTATTTGTAAAACAAGAATTAGGTTTG_GCTCAATACTCATGCAATACCTACAACTGAGGATTCTTCCCAGGGGGACCGCAGCCTGTCAGCATGGCTCAAGAGTGTGGGAACTACAAAATCCAGCCTGGGAAGGT_GGTTGTGT_TCATCAAGCCCACCTTCCCATTCTGCAG_GAGGACCCAAGAGAT_CCTCAGTCAACTGCCCATCAAACA_AGGGCTTCTGGAATTTGTCGAGAT_CACA_ACCACCAACCACACTA_AC___AAGATTCAAGATTATT_TGGA_ACAGCTCACGGGAGCAAGAATGG_GGCCTTGAGTCTTTATCAGTAAAGATTGTATAGGCAGATGCAGTGA_TCTAGTCACTATGC_AATAGAGTGGGGAACTGC_TGACGCGGCTAAAGCAGATTG__GAGTTTTGCAGTA_ACCAC_AG_A__GCAGGCCACACGCTGACTTTCCTCCTCAAGAGCTGGATGGCATTGCATGTGATGACAG_CACT__TCCTGGTGGATGAATTTGGGGGGCACAAACAGCT_TTTTTCTTCTTTTGCCTCAGTATTTAAAAGTGGATCAACTTGCTCTTGAC_CACGGGGCCAAGAAGGTTGATGGGCCATC_TTGGTTTTCTTCTTGATGTG_CTCTTTGGTTTTCAGAAGATTGTGACAAGTTCTG_GCCTAG_G_ATTCGCTCACTCACCCTCAATTGTTCTTTCTCT_T_TGGCACGCGTTTCTTACTGTTCTCCACGTGTCGGCATGCCTCTGCCTCTAAGCCAGTGT_TTTTCAA_C_TATGTT_TCTACA_GA__CTCCTTCTCCACAATGATGAATCCTTGGTTGGTTTTCTGCT_ACTGCCCAT_T_AGCT_GAAATCATTTTTCCTCTT_GACTTTGT_GGAGTTGGTGTTATGAAA

In [8]:
global_align(prot_seq1, prot_seq2, 10, blo_dict)

Max Score:  3594

Global Alignment: 
MSSSCWLLLSLVAVATAQSLIEEKAESFLNKFNQEAEDLSYQSSLASWNYNTNITEENAQKMNEAAAKWSAFYEEQSKIAQNFSLQEIQNATIKRQLKALQQSGSSALSPDKNKQLNTILNTMSTIYSTGKVCNSMNPQECFLLEPGLDEIMATSTDYNRRLWAWEGWRAEVGKQLRPLYEEYVVLKNEMARANNYEDYGDYWRGDYEAEGVEGYNYNRNQLIEDVENTFKEIKPLYEQLHAYVRTKLMEVYPSYISPTGCLPAHLLGDMWGRFWTNLYPLTTPFLQKPNIDVTDAMVNQSWDAERIFKEAEKFFVSVGLPQMTPGFWTNSMLTEPGDDRKVVCHPTAWDLGHGDFRIKMCTKVTMDNFLTAHHEMGHIQYDMAYAKQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLPSNFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFQDKIPREQWTKKWWEMKREIVGVVEPLPHDETYCDPASLFHVSNDYSFIRYYTRTIYQFQFQEALCQAAKHDGPLHKCDISNSTEAGQKLLNMLSLGNSGPWTLALENVVGSRNMDVKPLLNYFQPLFVWLKEQNRNSTVGWSTDWSPYADQSIKVRISLKSALGKNAYEWTDNEMYLFRSSVAYAMREYFSREKNQTVPFGEADVWVSDLKPRVSFNFFVTSPKNVSDIIPRSEVEEAIRMSRGRINDIFGLNDNSLEFLGIYPTLKPPYEPPVTIWLIIFGVVMGTVVVGIVILIVTGIKGRKKKNETKREENPYDSMDIGKGESNAGFQNSDDAQTSF
MSSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDNPQECLLLEPGLNEIMANSLD

### Question 2 - Local Alignment

* Default gap penalty = 10
* Score Matirx used = BLOSUM62 - protein / DNAfull - DNA

In [13]:
def local(seq1, seq2, gap_penalty, blosum_dict):

    smatrixToDict = {}
    direction = 0 
    runningMaxScore = 0
    maxLocation = 0,0
    
    smatrix = np.zeros((len(seq1)+2, len(seq2)+2), dtype = np.object)
    
    toprow = smatrix[0]
    seq2 = " "+ " " + seq2
    seq1 = " "+ seq1
    for i in range(len(seq2)):
        toprow[i] = seq2[i] 

    for k in range(len(seq1)): 
        smatrix[k+1][0] = seq1[k]

    for rowi in range(2, len(seq1)+1): 
        for colj in range(2, len(seq2)):  

            blosumScoreKey = (smatrix[rowi][0],smatrix[0][colj])

#             print(smatrix)
            scorefromabove = int(smatrix[rowi-1][colj]) -(gap_penalty)
            scorefromleft = int(smatrix[rowi][colj-1]) -(gap_penalty)
            scorefromdiag= (smatrix[rowi-1][colj-1]) + int(blosum_dict.get(blosumScoreKey))
            zero = 0
            
            maxof4scores = max(scorefromabove, scorefromleft, scorefromdiag, zero)

            smatrix[rowi][colj] = maxof4scores

            if(maxof4scores == scorefromabove): # gap goes in seq1 horiz aa
                direction = 1
            if(maxof4scores == scorefromleft): # gap goes in seq2 vert aa 
                direction = 3
            if(maxof4scores == scorefromdiag):
                direction = 2
            if(maxof4scores == 0):
                direction = 0
                
            smatrixToDict[rowi, colj] = direction, maxof4scores 
                    
            if(maxof4scores > runningMaxScore):
                runningMaxScore = max(runningMaxScore, maxof4scores)     
                maxLocation = rowi, colj    
 
    maxScore = smatrix[maxLocation]           
    location = maxLocation       
    
    print(smatrix)

    directionANDscore = smatrixToDict[location]
    direction, score = (directionANDscore[0], directionANDscore[1])
    newseq1 = []
    newseq2 = []
     
     
    while 1:      
        if(direction == 1):  #score from above
            newlocation = ((location[0])-1, location[1])  #location = [row, column] of score
            newseq2.append("_")
            newseq1.append(smatrix[location[0]][0])  #smatrix[locationtuple(row, col)][0]
                                                         
        if(direction == 3):    #score from left
            newseq1.append("_")   
            newlocation = (location[0], (location[1])-1)
            newseq2.append(smatrix[0][location[1]])             

        if(direction == 2): #score from diagonal                      
            newlocation = ((location[0])-1, (location[1])-1)
            newseq1.append(smatrix[location[0]][0])                    
            newseq2.append(smatrix[0][location[1]])  

#         print(newseq1)
#         print(newseq2)
#         print(score)
        location = newlocation
        if(location[0] == 2 or location[1] == 2 or score <= 0):
            break
        directionANDscore = smatrixToDict[location]
        score = (directionANDscore[1]) 
        direction = directionANDscore[0]
    
    newseq1 = "".join(newseq1[::-1])
    newseq2 = "".join(newseq2[::-1])
    
    print("Max Score: ", maxScore)
    print("Location: ", maxLocation)
    print("\n Local Alignment: ")
    print(newseq1)
    print(newseq2)

In [14]:
local(dna_seq1, dna_seq2, 10, dna_dict)

[[' ' ' ' 'A' ... 'T' 'T' 'C']
 [' ' 0 0 ... 0 0 0]
 ['T' 0 0 ... 5 5 0]
 ...
 ['T' 0 0 ... 20 10 7]
 ['G' 0 0 ... 17 16 6]
 ['G' 0 0 ... 23 13 12]]
Max Score:  95
Location:  (554, 509)

 Local Alignment: 
ACAAG_ATTCAAGAT_TATTTGGAACAGCTCACGGGAGCAAGAATGGGGCCTTGAGTCTTTATCAGTAAAGATTGTATAGGCAGATGCAGTGATCTAGTCACTATGCAATAGAGTGGGGAACTGCTGACGCGGCTAAAGCAGATTGGAGTTTTGCAGTAACCACAGAGCAGGCCACACGCTGACTTTCCTCCTCAAGAGCTGGATGGCATTGCATG_T_GATGACAGCACTTCCTGGTGGATGAATTTGGGGGGCACAA_ACAGC
AGAAGCAGGCAAGGTATACTT_GAA_GGCTCCCATGATTCTGAATGGAG___TGTGT_GTTATATGGAAGGGCTGGAT_TG_ATCTCCACAGAT_TGGATGGTATG_GGTTGCCTGGAG_TTTGATGA_GGAGCGAGCCCAG_CAGGA_AGATGCA_TTAGCACA_A_CAGGCC_TTTGAAGAGGCT_C_GAAGAAGAACTCGA__GAATTTGAGGATAGA_GACAGGTC_TCACCG_GGAGGAA_ATGGAGGCAAGAAGACAGC


In [11]:
local(prot_seq1, prot_seq2, 10, blo_dict)

Max Score:  3594
Location:  (806, 806)

 Local Alignment: 
SSSCWLLLSLVAVATAQSLIEEKAESFLNKFNQEAEDLSYQSSLASWNYNTNITEENAQKMNEAAAKWSAFYEEQSKIAQNFSLQEIQNATIKRQLKALQQSGSSALSPDKNKQLNTILNTMSTIYSTGKVCNSMNPQECFLLEPGLDEIMATSTDYNRRLWAWEGWRAEVGKQLRPLYEEYVVLKNEMARANNYEDYGDYWRGDYEAEGVEGYNYNRNQLIEDVENTFKEIKPLYEQLHAYVRTKLMEVYPSYISPTGCLPAHLLGDMWGRFWTNLYPLTTPFLQKPNIDVTDAMVNQSWDAERIFKEAEKFFVSVGLPQMTPGFWTNSMLTEPGDDRKVVCHPTAWDLGHGDFRIKMCTKVTMDNFLTAHHEMGHIQYDMAYAKQPFLLRNGANEGFHEAVGEIMSLSAATPKHLKSIGLLPSNFQEDNETEINFLLKQALTIVGTLPFTYMLEKWRWMVFQDKIPREQWTKKWWEMKREIVGVVEPLPHDETYCDPASLFHVSNDYSFIRYYTRTIYQFQFQEALCQAAKHDGPLHKCDISNSTEAGQKLLNMLSLGNSGPWTLALENVVGSRNMDVKPLLNYFQPLFVWLKEQNRNSTVGWSTDWSPYADQSIKVRISLKSALGKNAYEWTDNEMYLFRSSVAYAMREYFSREKNQTVPFGEADVWVSDLKPRVSFNFFVTSPKNVSDIIPRSEVEEAIRMSRGRINDIFGLNDNSLEFLGIYPTLKPPYEPPVTIWLIIFGVVMGTVVVGIVILIVTGIKGRKKKNETKREENPYDSMDIGKGESNAGFQNSDDAQTSF
SSSSWLLLSLVAVTAAQSTIEEQAKTFLDKFNHEAEDLFYQSSLASWNYNTNITEENVQNMNNAGDKWSAFLKEQSTLAQMYPLQEIQNLTVKLQLQALQQNGSSVLSEDKSKRLNTILNTMSTIYSTGKVCNPDN